In [1]:
#@formatter:off
%load_ext autoreload
%autoreload 2
#@formatter:on

In [2]:
import os
import random
import string

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import TextVectorization

In [3]:
class ModelConfig:
    M_VOCAB_SZ = 20000
    M_MAX_LEN = 80
    M_ATT_HEADS = 2
    M_DIM_EMB = 256
    M_DIM_FFN = 256
    M_WARMUP_STEPS = 2000

class TrainingConfig:
    T_BATCH_SIZE = 256
    T_EPOCHS = 5

data_files = ["data_test_wikitext/wiki.train.tokens"]

In [4]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
    return tf.tile(mask, mult)


class Transformer(layers.Layer):
    def __init__(self, embedding_dim, num_att_heads, state_dims, dropout_rate=0.1):
        super(Transformer, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_att_heads = num_att_heads
        self.state_dims = state_dims
        self.dropout_rate = dropout_rate
        self.attention = layers.MultiHeadAttention(num_att_heads, embedding_dim)
        self.feed_forward = keras.Sequential([
            layers.Dense(state_dims, activation="relu"),
            layers.Dense(embedding_dim)
        ])
        self.norm1, self.norm2 = layers.LayerNormalization(epsilon=1e-6), layers.LayerNormalization(epsilon=1e-6)
        self.drop1, self.drop2 = layers.Dropout(dropout_rate), layers.Dropout(dropout_rate)

    def call(self, inputs):
        inp_shape = tf.shape(inputs)
        batch_sz, seq_len = inp_shape[0], inp_shape[1]
        causal_mask = causal_attention_mask(batch_sz, seq_len, seq_len, tf.bool)
        attention_out = self.attention(inputs, inputs, attention_mask=causal_mask)
        attention_out = self.drop1(attention_out)
        out1 = self.norm1(inputs + attention_out)
        feed_forward_out = self.feed_forward(out1)
        feed_forward_out = self.drop2(feed_forward_out)
        return self.norm2(out1 + feed_forward_out)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embedding_dim": self.embedding_dim,
            "num_att_heads": self.num_att_heads,
            "state_dims": self.state_dims,
            "dropout_rate": self.dropout_rate,
        })
        return config


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        pos = tf.range(start=0, limit=max_len, delta=1)
        pos = self.pos_emb(pos)
        x = self.token_emb(x)
        return x + pos

    def get_config(self):
        config = super().get_config()
        config.update({
            "max_len": self.max_len,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

In [5]:
class WarmupScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, embedding_dim, warmup_steps):
        super(WarmupScheduler, self).__init__()
        self.emb_dim = tf.cast(embedding_dim, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.emb_dim) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        config = super().get_config()
        config.update({
            "emb_dim": self.emb_dim,
            "warmup_steps": self.warmup_steps,
        })
        return config

In [6]:
def create_model(model_config: ModelConfig):
    # TODO: Remove this if we're using tokenizer
    embedding = TokenAndPositionEmbedding(model_config.M_MAX_LEN, model_config.M_VOCAB_SZ, model_config.M_DIM_EMB)
    transformer = Transformer(model_config.M_DIM_EMB, model_config.M_ATT_HEADS, model_config.M_DIM_FFN)

    l_input = layers.Input(shape=(model_config.M_MAX_LEN,), dtype=tf.int32)
    l_emb = embedding(l_input)
    l_trans = transformer(l_emb)
    l_output = layers.Dense(model_config.M_VOCAB_SZ)(l_trans)

    m = keras.Model(inputs=l_input, outputs=[l_output, l_trans])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    learning_rate = WarmupScheduler(model_config.M_DIM_EMB, model_config.M_WARMUP_STEPS)
    optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    m.compile("adam", loss=[loss_fn, None])
    return m

In [7]:
def create_dataset(file_pth, batch_sz, buf_sz=1000, shuffle=True):
    # Shuffle the data and create batches
    if shuffle:
        random.shuffle(file_pth)
    ds = tf.data.TextLineDataset(file_pth)
    ds = ds.shuffle(buffer_size=buf_sz)
    ds = ds.batch(batch_sz)
    return ds


def create_tokenizer(dataset, max_vocab_size, max_seq_len):
    def preprocess_txt(input_string):
        # Preprocessing for word-level model
        s1 = tf.strings.lower(input_string)
        return tf.strings.regex_replace(s1, f"([{string.punctuation}])", r" \1")

    # Vectorization of the data
    vectorize = TextVectorization(
        standardize=preprocess_txt,
        max_tokens=max_vocab_size - 1,
        output_mode="int",
        output_sequence_length=max_seq_len + 1,
    )
    vectorize.adapt(dataset)
    vocab = vectorize.get_vocabulary()
    return vectorize, vocab


config_model = ModelConfig()
config_training = TrainingConfig()
# Read in the data and create the dataset
dataset = create_dataset(data_files, config_training.T_BATCH_SIZE)
# Create the tokenizer
tokenizer, vocab = create_tokenizer(dataset, config_model.M_VOCAB_SZ, config_model.M_MAX_LEN)


def create_sequences(txt):
    txt = tf.expand_dims(txt, -1)
    txt_tok = tokenizer(txt)
    return txt_tok[:, :-1], txt_tok[:, 1:]


dataset = dataset.map(create_sequences).prefetch(tf.data.AUTOTUNE)

2022-05-16 19:30:59.855033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-16 19:30:59.882207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-16 19:30:59.882363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-16 19:30:59.882722: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
for d in dataset.take(1):
    print(d)

(<tf.Tensor: shape=(256, 80), dtype=int64, numpy=
array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [  13,   13,   13, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [  13, 2367,   42, ...,    0,    0,    0],
       [ 621,   50,    2, ...,   10,  917,    3]])>, <tf.Tensor: shape=(256, 80), dtype=int64, numpy=
array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [  13,   13,  732, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [2367,   42,  805, ...,    0,    0,    0],
       [  50,    2,    7, ...,  917,    3, 7707]])>)


In [9]:
class GenerationCallback(keras.callbacks.Callback):
    def __init__(self, max_tokens, seq_len, start_tokens, index_to_word, top_k=10, print_every=1):
        self.max_tokens = max_tokens
        self.seq_len = seq_len
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = list(self.start_tokens)
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = self.seq_len - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:self.seq_len]
                sample_index = self.seq_len - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join([self.detokenize(_) for _ in self.start_tokens + tokens_generated])
        print(f"Generated:\n{txt}\n")

    @staticmethod
    def create(start_prompt, seq_len, vocabulary, gen_len=100):
        # Tokenize starting prompt
        word_to_index = {word: index for index, word in enumerate(vocabulary)}
        prompt_tokens = [word_to_index.get(_, 1) for _ in start_prompt.lower().split()]
        return GenerationCallback(gen_len, seq_len, prompt_tokens, vocabulary)

In [10]:


def create_callbacks(base_dir, model, defaults: list = None):
    dir_models = os.path.join(base_dir, model.name)
    os.makedirs(dir_models, exist_ok=True)
    print(f'# Model Dir: {dir_models}')

    path_csv = os.path.join(dir_models, 'history.csv')
    print(" - History CSV:", path_csv)

    path_ckp = os.path.join(dir_models, 'checkpoints.h5')
    print(" - Checkpoint:", path_ckp)

    path_tb = os.path.join(dir_models, "logs")
    tb_file_writer = tf.summary.create_file_writer(path_tb)
    callbacks = [] if defaults is None else defaults
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=path_tb, histogram_freq=1))
    callbacks.append(tf.keras.callbacks.CSVLogger(path_csv, separator=",", append=True))
    # callbacks.append(tf.keras.callbacks.ModelCheckpoint(path_ckp,
    #                                                     monitor='loss',
    #                                                     save_best_only=True,
    #                                                     mode='auto',
    #                                                     verbose=0))
    os.makedirs(dir_models, exist_ok=True)
    return callbacks, tb_file_writer

In [11]:


model = create_model(config_model)
callbacks, tb_file_writer = create_callbacks("logs", model)
gen_callback = GenerationCallback.create("a day in the life", config_model.M_MAX_LEN, vocab)
callbacks.append(gen_callback)

# Model Dir: logs/model
 - History CSV: logs/model/history.csv
 - Checkpoint: logs/model/checkpoints.h5


In [12]:
# wandb.tensorboard.patch(root_logdir="logs")
# wandb.init(project='transformer')
model.fit(dataset, verbose=2, epochs=config_training.T_EPOCHS, callbacks=callbacks)

Epoch 1/5


AttributeError: in user code:

    File "/home/acozma/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/acozma/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/acozma/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/acozma/.local/lib/python3.8/site-packages/keras/engine/training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/acozma/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/home/acozma/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py", line 6

In [ ]:
model.save("model_warmup.h5")